# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import warnings
warnings.filterwarnings('ignore')

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_to_load = "../output_data/cities.csv"
# Read Purchasing File and store into Pandas data frame
cities_data_1 = pd.read_csv(file_to_load)
cities_data_1.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Norman Wells,65.2820,-126.8329,-12.96,70,100,18.41,CA,1644204055
1,1,Pangnirtung,66.1451,-65.7125,-38.09,96,100,4.47,CA,1644204397
2,2,Lavrentiya,65.5833,-171.0000,-15.63,95,56,5.19,RU,1644204530
3,3,Camaçari,-12.6975,-38.3242,82.17,89,95,3.67,BR,1644204579
4,4,Faanui,-16.4833,-151.7500,81.01,75,100,20.13,PF,1644204048


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#set data for gmaps
locations = cities_data_1[["Lat", "Lng"]]
humidity = cities_data_1['Humidity'].astype(int)
maxhumidity = humidity.max()

# # Plot Heatmap
center_is = (40.46,3.75)
figure = gmaps.figure(layout={
        'width': '1200px',
        'height': '900px',
        'padding': '3px',
        'border': '1px solid black'
})
figure = gmaps.figure()
figure = gmaps.figure(zoom_level = 1.5,
                      center = center_is)
# # Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=maxhumidity, point_radius=2)
# # Add layer
figure.add_layer(heat_layer)
# # Display figure
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#set colors for the while loop responses
class bcolors:
    OK = '\033[92m' #GREEN
    WARNING = '\033[93m' #YELLOW
    FAIL = '\033[91m' #RED
    RESET = '\033[0m' #RESET COLOR

In [20]:
#set user input
print(f'---Please enter a value between 70 and 90 (no decimals)---\n')
while True:
    try:
        min_t = int(input('What is the lowest temperature you desired?'))
    except ValueError:
        print(f'{bcolors.WARNING}Value must be a number.{bcolors.RESET}')
    else:
        if 69 < min_t < 91:
            break
        else:
            print(f'{bcolors.FAIL}Value must be between 70 and 90!{bcolors.RESET}')
while True:
    try:
        max_t = int(input('What about the hotest?'))
    except ValueError:
        print(f'{bcolors.WARNING}Value must be a number.{bcolors.RESET}')
    else:
        if min_t+1 < max_t < 91:
            break
        else:
            print(f'{bcolors.FAIL}Value must be between {min_t+2} and 95{bcolors.RESET}')
print(f'---Please enter a value between 0.0 and 10.0---' )
while True:
    try:
        wind_s = float(input('How windy do you like your vacation?'))
    except ValueError:
        print(f'{bcolors.WARNING}Value must be a number.{bcolors.RESET}')
    else:
        if 0.0 <= wind_s <= 10.0:
            break
        else:
            print(f'{bcolors.FAIL}Value must be between 0.0 and 10.0!{bcolors.RESET}')

---Please enter a value between 70 and 90 (no decimals)---

What is the lowest temperature you desired?70
What about the hotest?88
---Please enter a value between 0.0 and 10.0---
How windy do you like your vacation?9


In [21]:
vacation_df = cities_data_1[(cities_data_1['Max Temp'].between(min_t,max_t)) & (cities_data_1['Wind Speed']<=wind_s) & (cities_data_1['Humidity']<=45)& (cities_data_1["Cloudiness"] == 0)]
vacation_df.reset_index(inplace=True)
del vacation_df['index']
message =print(f'Threre are(is) --{len(vacation_df)}-- citie(s) based on you criteria:\n'
                f'These are your selections!\n'
                f'Temperatures between {min_t} and {max_t}.\n'
                f'Wind speed of {wind_s}and Zero cloudiness'
              )
print(message)
vacation_df

Threre are(is) --2-- citie(s) based on you criteria:
These are your selections!
Temperatures between 70 and 88.
Wind speed of 9.0and Zero cloudiness
None


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,170,New Norfolk,-42.7826,147.0587,81.66,28,0,3.00,AU,1644204764
1,454,Lokoja,7.8024,6.7430,72.28,21,0,1.99,NG,1644205057


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
hotel_df = []
for x in range(len(vacation_df)):
    lat = vacation_df.loc[x]['Lat']
    lng = vacation_df.loc[x]['Lng']
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key}
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
     
    try:
        hotel_df.append(response['results'][0]['name'])
    except:
        hotel_df.append("")

In [23]:
#dispalyinf list of hotels
vacation_df["Hotel Name"] = hotel_df
vacation_df = vacation_df.dropna(how='any')
vacation_df['Hotel Name']

0    New Norfolk
1         Lokoja
Name: Hotel Name, dtype: object

In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]
locations = vacation_df[["Lat", "Lng"]]

In [25]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
figure.add_layer(markers)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))